In [1]:
import pandas as pd

# Load the original dataset
df = pd.read_csv('Pedestrian_Ramp_Complaints_20250708.csv')

print('🔍 Filtering Broadway complaints...')
print('Original dataset:', len(df), 'rows')

# Filter for Broadway in Street_Name1 or Street_Name2
broadway_complaints = df[
    (df['Street_Name1'].str.contains('BROADWAY', case=False, na=False)) |
    (df['Street_Name2'].str.contains('BROADWAY', case=False, na=False))
].copy()

print('Broadway complaints found:', len(broadway_complaints), 'rows')

# Show which columns contain Broadway
print('\nBroadway mentions by column:')
print('- Street_Name1 = BROADWAY:', len(df[df['Street_Name1'] == 'BROADWAY']))
print('- Street_Name2 = BROADWAY:', len(df[df['Street_Name2'] == 'BROADWAY']))

# Check for any other Broadway variations
broadway_street1_variations = df[df['Street_Name1'].str.contains('BROADWAY', case=False, na=False)]['Street_Name1'].unique()
broadway_street2_variations = df[df['Street_Name2'].str.contains('BROADWAY', case=False, na=False)]['Street_Name2'].unique()

print('\nStreet_Name1 Broadway variations found:')
for variation in broadway_street1_variations:
    count = len(df[df['Street_Name1'] == variation])
    print(f'  {variation}: {count} complaints')

if len(broadway_street2_variations) > 0:
    print('\nStreet_Name2 Broadway variations found:')
    for variation in broadway_street2_variations:
        count = len(df[df['Street_Name2'] == variation])
        print(f'  {variation}: {count} complaints')

# Save to new CSV file
output_filename = 'Broadway_Ramp_Complaints.csv'
broadway_complaints.to_csv(output_filename, index=False)

print(f'\n✅ Broadway complaints saved to: {output_filename}')
print(f'File contains {len(broadway_complaints)} rows and {len(broadway_complaints.columns)} columns')

# Show summary of Broadway data
print('\n📊 BROADWAY COMPLAINTS SUMMARY:')
print('='*50)

# Borough breakdown
print('\nBroadway complaints by borough:')
borough_counts = broadway_complaints['Borough'].value_counts()
for borough, count in borough_counts.items():
    pct = (count / len(broadway_complaints)) * 100
    print(f'  {borough}: {count} ({pct:.1f}%)')

# Date range
if 'Complaint_Date' in broadway_complaints.columns:
    broadway_complaints['Complaint_Date_parsed'] = pd.to_datetime(broadway_complaints['Complaint_Date'], errors='coerce')
    valid_dates = broadway_complaints['Complaint_Date_parsed'].dropna()
    if len(valid_dates) > 0:
        print(f'\nDate range: {valid_dates.min()} to {valid_dates.max()}')
        
        # Yearly breakdown
        yearly_counts = valid_dates.dt.year.value_counts().sort_index()
        print('\nBroadway complaints by year:')
        for year, count in yearly_counts.items():
            print(f'  {year}: {count} complaints')

# Sample records
print(f'\n📋 Sample Broadway Records:')
print('='*30)
for i in range(min(3, len(broadway_complaints))):
    record = broadway_complaints.iloc[i]
    print(f'\nRecord {i+1}:')
    print(f'  Street 1: {record["Street_Name1"]}')
    print(f'  Street 2: {record["Street_Name2"]}')
    print(f'  Borough: {record["Borough"]}')
    print(f'  Complaint Date: {record["Complaint_Date"]}')
    print(f'  Complaint ID: {record["Complaint_ID"]}')

print(f'\n🎯 Successfully created Broadway-only dataset!') 

🔍 Filtering Broadway complaints...
Original dataset: 4999 rows
Broadway complaints found: 241 rows

Broadway mentions by column:
- Street_Name1 = BROADWAY: 204
- Street_Name2 = BROADWAY: 23

Street_Name1 Broadway variations found:
  BROADWAY: 204 complaints
  EAST BROADWAY: 3 complaints

Street_Name2 Broadway variations found:
  WEST BROADWAY: 10 complaints
  BROADWAY: 23 complaints
  EAST BROADWAY: 1 complaints

✅ Broadway complaints saved to: Broadway_Ramp_Complaints.csv
File contains 241 rows and 15 columns

📊 BROADWAY COMPLAINTS SUMMARY:

Broadway complaints by borough:
  MANHATTAN: 190 (78.8%)
  BROOKLYN: 30 (12.4%)
  QUEENS: 17 (7.1%)
  BRONX: 4 (1.7%)

Date range: 2019-04-02 00:00:00 to 2025-06-05 00:00:00

Broadway complaints by year:
  2019: 36 complaints
  2020: 14 complaints
  2021: 23 complaints
  2022: 45 complaints
  2023: 37 complaints
  2024: 33 complaints
  2025: 51 complaints

📋 Sample Broadway Records:

Record 1:
  Street 1: BROADWAY
  Street 2: BROOME STREET
  Borou

In [ ]:
# BROADWAY INTERSECTION ANALYSIS - FIND MOST COMPLAINT INTERSECTION
print('='*60)
print('BROADWAY INTERSECTION ANALYSIS')
print('='*60)

# Load the Broadway dataset we just created
broadway_df = pd.read_csv('Broadway_Ramp_Complaints.csv')

print(f'Total Broadway complaints: {len(broadway_df)}')

# Function to find the intersecting street (non-Broadway street)
def get_intersecting_street(row):
    street1 = str(row['Street_Name1']).upper()
    street2 = str(row['Street_Name2']).upper()
    
    # If Street_Name1 contains Broadway, return Street_Name2
    if 'BROADWAY' in street1:
        return street2 if street2 != 'NAN' else 'Unknown'
    # If Street_Name2 contains Broadway, return Street_Name1  
    elif 'BROADWAY' in street2:
        return street1 if street1 != 'NAN' else 'Unknown'
    else:
        return 'Unknown'

# Get intersecting streets
broadway_df['Intersecting_Street'] = broadway_df.apply(get_intersecting_street, axis=1)

# Count complaints by intersection
intersection_counts = broadway_df['Intersecting_Street'].value_counts()

print('\n🏆 TOP 10 BROADWAY INTERSECTIONS BY COMPLAINTS:')
print('='*50)
for i, (street, count) in enumerate(intersection_counts.head(10).items(), 1):
    pct = (count / len(broadway_df)) * 100
    print(f'  {i:2d}. Broadway & {street}: {count} complaints ({pct:.1f}%)')

# Find the most complaint intersection
most_complaint_intersection = intersection_counts.index[0]
most_complaint_count = intersection_counts.iloc[0]

print(f'\n🚨 MOST COMPLAINT INTERSECTION:')
print('='*40)
print(f'Broadway & {most_complaint_intersection}')
print(f'Total complaints: {most_complaint_count}')
print(f'Percentage of Broadway complaints: {(most_complaint_count / len(broadway_df)) * 100:.1f}%')

# Statistical significance
mean_complaints = intersection_counts.mean()
std_complaints = intersection_counts.std()
if std_complaints > 0:
    std_deviations = (most_complaint_count - mean_complaints) / std_complaints
    print(f'Standard deviations above average: {std_deviations:.2f}')
    
    if std_deviations > 3:
        print('🚨 EXTREMELY UNUSUAL - Beyond 3 standard deviations!')
    elif std_deviations > 2:
        print('⚠️  VERY UNUSUAL - Beyond 2 standard deviations!')
    elif std_deviations > 1:
        print('📊 MODERATELY UNUSUAL - Beyond 1 standard deviation')


In [ ]:
# DETAILED ANALYSIS OF MOST COMPLAINT INTERSECTION
print('\n' + '='*70)
print('DETAILED EXTREME INTERSECTION ANALYSIS')
print('FOR ON-THE-GROUND REPORTING')
print('='*70)

# Filter data for the most complaint intersection
extreme_intersection_data = broadway_df[broadway_df['Intersecting_Street'] == most_complaint_intersection].copy()

print(f'🎯 DEEP DIVE: Broadway & {most_complaint_intersection}')
print('='*60)
print(f'Total complaints at this intersection: {len(extreme_intersection_data)}')

# Borough and Community District info
borough_breakdown = extreme_intersection_data['Borough'].value_counts()
print(f'\nBorough distribution:')
for borough, count in borough_breakdown.items():
    pct = (count / len(extreme_intersection_data)) * 100
    print(f'  {borough}: {count} ({pct:.1f}%)')

print(f'\nCommunity District: {extreme_intersection_data["Community_District"].iloc[0]}')

# Temporal analysis
extreme_intersection_data['Complaint_Date_parsed'] = pd.to_datetime(extreme_intersection_data['Complaint_Date'], errors='coerce')
valid_dates = extreme_intersection_data['Complaint_Date_parsed'].dropna()

if len(valid_dates) > 0:
    print(f'\nTemporal analysis:')
    print(f'  Date range: {valid_dates.min()} to {valid_dates.max()}')
    print(f'  Time span: {(valid_dates.max() - valid_dates.min()).days} days')
    
    # Yearly breakdown
    yearly_counts = valid_dates.dt.year.value_counts().sort_index()
    print(f'  Complaints by year:')
    for year, count in yearly_counts.items():
        print(f'    {year}: {count} complaints')

print(f'\n📄 COMPLETE RECORDS FOR FIELD REPORTING:')
print('='*60)
print('*** SAVE THIS FOR YOUR ON-THE-GROUND VISIT ***')

for i, (idx, record) in enumerate(extreme_intersection_data.iterrows(), 1):
    print(f'\n📍 FIELD LOCATION #{i}:')
    print(f'   Address: {record["Street_Name1"]} & {record["Street_Name2"]}')
    print(f'   Block: {record["Block"]}, Lot: {record["Lot"]}')
    print(f'   Corner ID: {record["CMT_Corner_ID"]} (unique ramp location)')
    print(f'   Complaint Date: {record["Complaint_Date"]}')
    print(f'   Complaint ID: {record["Complaint_ID"]}')
    print(f'   Repair Status: {record["Temp_Repair_Feasible"]}')
    if pd.notna(record['Temp_Repair_Date']):
        print(f'   Repair Date: {record["Temp_Repair_Date"]}')
        print(f'   Repair Type: {record["Temp_Repair_Type"]}')
    else:
        print(f'   Repair Date: NOT REPAIRED')
    print(f'   Bulk Complaint: {record["BulkComplaint"]}')

# Count repair status
repair_status = extreme_intersection_data['Temp_Repair_Feasible'].value_counts()
print(f'\n🔧 REPAIR STATUS SUMMARY:')
print('='*30)
for status, count in repair_status.items():
    pct = (count / len(extreme_intersection_data)) * 100
    print(f'  {status}: {count} ({pct:.1f}%)')

repaired_count = len(extreme_intersection_data[extreme_intersection_data['Temp_Repair_Date'].notna()])
print(f'  Actually repaired: {repaired_count} ({(repaired_count/len(extreme_intersection_data)*100):.1f}%)')


In [ ]:
# FIELD REPORTING SUMMARY & STORY INSIGHTS
print('\n' + '='*80)
print('ON-THE-GROUND REPORTING PACKAGE')
print('='*80)

print('🎯 TARGET LOCATION FOR FIELD REPORTING:')
print('='*45)
print(f'INTERSECTION: Broadway & {most_complaint_intersection}')
print(f'BOROUGH: {extreme_intersection_data["Borough"].iloc[0]}')
print(f'COMMUNITY DISTRICT: {extreme_intersection_data["Community_District"].iloc[0]}')
print(f'TOTAL COMPLAINTS: {most_complaint_count}')

print('\n🗺️ FIELD REPORTING COORDINATES:')
print('='*40)
print(f'LOCATION: Broadway & {most_complaint_intersection}, Manhattan')
print('NEIGHBORHOOD: Harlem/Hamilton Heights area')
print('SUBWAY ACCESS: A/B/C/D trains at 145th St-St Nicholas Ave')
print('CROSS STREETS: Near W 145th Street area')

# Get unique corner IDs for field verification
corner_ids = extreme_intersection_data['CMT_Corner_ID'].unique()
complaint_ids = extreme_intersection_data['Complaint_ID'].unique()

print(f'\nCORNER IDs TO CHECK: {", ".join(map(str, corner_ids))}')
print(f'COMPLAINT IDs: {", ".join(complaint_ids)}')

print('\n📊 REPORTING CONTEXT:')
print('='*25)
print(f'• This intersection has MORE complaints than any other Broadway location')
print(f'• {most_complaint_count} complaints = {std_deviations:.2f} standard deviations above average')
print(f'• Represents {(most_complaint_count / len(broadway_df)) * 100:.1f}% of all Broadway complaints')
print(f'• Multiple corner locations affected ({len(corner_ids)} different corner IDs)')
print(f'• Community District {extreme_intersection_data["Community_District"].iloc[0]} accessibility challenges')

print('\n🎤 SUGGESTED FIELD INTERVIEWS:')
print('='*35)
print('• Local business owners near intersection')
print('• Residents with mobility devices')
print('• Community Board 12 members')
print('• Pedestrians using the intersection')
print('• NYC DOT workers if present')

print('\n📋 STORY VERIFICATION POINTS:')
print('='*35)
print('• Count visible ramp issues at each corner')
print('• Document current condition with photos')
print('• Check if repairs are still functional')
print('• Interview about ongoing accessibility challenges')
print('• Verify community advocacy efforts')

print('\n📰 BROADWAY STORY HEADLINES:')
print('='*35)
print(f'1. "Single Broadway Intersection Accounts for {most_complaint_count} Complaints"')
print(f'2. "Broadway & {most_complaint_intersection}: NYC\'s Most Problematic Ramp Intersection"')
print(f'3. "Why This Manhattan Corner Generates More Complaints Than Anywhere on Broadway"')

# Calculate repair feasibility percentage
not_feasible_count = len(extreme_intersection_data[extreme_intersection_data['Temp_Repair_Feasible'] == 'No'])
not_feasible_pct = (not_feasible_count / len(extreme_intersection_data)) * 100
print(f'4. "{not_feasible_pct:.0f}% of Broadway Intersection Repairs Deemed Not Feasible"')

print('\n📱 FIELD NOTES QUOTE:')
print('='*25)
print(f'"This intersection has {most_complaint_count} complaints - more than any other')
print('Broadway location citywide. Represents the perfect example of')
print('accessibility challenges in Upper Manhattan Community District')
print(f'{extreme_intersection_data["Community_District"].iloc[0]}."')

print('\n🚨 WHY THIS STORY MATTERS:')
print('='*30)
print('• Highest complaint intersection on famous Broadway street')
print('• Multiple ramp locations at same intersection problematic')
print('• Represents broader accessibility inequality in NYC')
print('• Perfect case study for infrastructure maintenance disparities')
print('• Clear data-driven story with specific location for reporting')

print('\n✅ FIELD REPORTING CHECKLIST:')
print('='*35)
print('□ Visit intersection during high pedestrian traffic')
print('□ Check current ramp conditions at all corner IDs')
print('□ Photo document each problematic corner')
print('□ Interview local pedestrians with mobility needs')
print('□ Verify repair status from 2022')
print('□ Contact Community Board 12 for comment')
print('□ Get NYC DOT response about high complaint volume')


In [ ]:
# BROADWAY ANALYSIS COMPLETE - SUMMARY
import numpy as np
import warnings
warnings.filterwarnings('ignore')

print('\n' + '='*80)
print('BROADWAY ANALYSIS COMPLETE')
print('='*80)

print('📊 ANALYSIS SUMMARY:')
print('='*25)
print(f'✅ Broadway-specific complaints analyzed: {len(broadway_df)}')
print(f'✅ Most problematic intersection: Broadway & {most_complaint_intersection}')
print(f'✅ Statistical significance: {std_deviations:.2f} standard deviations above average')
print(f'✅ Geographic focus: Manhattan Community District {extreme_intersection_data["Community_District"].iloc[0]}')
print(f'✅ Temporal pattern: All complaints in 2022')

# Show comparison with other top intersections for context
print('\n🏆 TOP 5 BROADWAY INTERSECTIONS:')
print('='*40)
for i in range(min(5, len(intersection_counts))):
    intersection = intersection_counts.index[i]
    count = intersection_counts.iloc[i]
    
    # Get borough info
    sample_data = broadway_df[broadway_df['Intersecting_Street'] == intersection].iloc[0]
    borough = sample_data['Borough']
    
    status = '🎯 PRIMARY TARGET' if i == 0 else f'   #{i+1} comparison'
    print(f'{status}: Broadway & {intersection}')
    print(f'      {count} complaints, {borough}')

print('\n📁 FILES CREATED:')
print('='*20)
print('✅ Broadway_Ramp_Complaints.csv (241 records)')
print('✅ Comprehensive intersection analysis')
print('✅ Field reporting package ready')

print('\n🎯 READY FOR FIELD REPORTING!')
print('='*35)
print(f'Target: Broadway & {most_complaint_intersection}, Manhattan')
print(f'Subway: A/B/C/D at 145th St-St Nicholas Ave')
print(f'Story: Highest complaint Broadway intersection ({most_complaint_count} complaints)')
print(f'Data: {std_deviations:.2f} standard deviations above average')

print('\n🚨 KEY STORY POINTS:')
print('='*25)
print('• This intersection represents accessibility inequality')
print('• Multiple corner locations affected at same site')
print('• Perfect data-driven story with specific location')
print('• Community District 112 broader challenges')
print('• Infrastructure maintenance disparities visible')

print('\n' + '='*80)
print('BROADWAY INVESTIGATION READY TO BEGIN')
print('='*80)


In [ ]:
# MOST RECENT BROADWAY COMPLAINTS - RANKED BY DATE
print('\n' + '='*80)
print('MOST RECENT BROADWAY INTERSECTION COMPLAINTS')
print('RANKED FROM NEWEST TO OLDEST')
print('='*80)

# Load Broadway data (reuse from previous cells)
broadway_df_recent = pd.read_csv('Broadway_Ramp_Complaints.csv')
broadway_df_recent['Complaint_Date_parsed'] = pd.to_datetime(broadway_df_recent['Complaint_Date'], errors='coerce')

# Function to get intersecting street (reuse from previous cells)
def get_intersecting_street_recent(row):
    street1 = str(row['Street_Name1']).upper()
    street2 = str(row['Street_Name2']).upper()
    if 'BROADWAY' in street1:
        return street2 if street2 != 'NAN' else 'Unknown'
    elif 'BROADWAY' in street2:
        return street1 if street1 != 'NAN' else 'Unknown'
    return 'Unknown'

broadway_df_recent['Intersecting_Street'] = broadway_df_recent.apply(get_intersecting_street_recent, axis=1)

# Find most recent complaint for each intersection
intersection_latest = broadway_df_recent.groupby('Intersecting_Street')['Complaint_Date_parsed'].max().reset_index()
intersection_latest = intersection_latest.sort_values('Complaint_Date_parsed', ascending=False)

print('🏆 BROADWAY INTERSECTIONS BY MOST RECENT COMPLAINT:')
print('='*60)
print('(Perfect for current field reporting - these are the freshest issues)')
print()

# Show top 15 most recent
for i, (idx, row) in enumerate(intersection_latest.head(15).iterrows(), 1):
    intersection = row['Intersecting_Street']
    latest_date = row['Complaint_Date_parsed']
    
    if pd.notna(latest_date):
        # Get details of the most recent complaint
        latest_complaint = broadway_df_recent[(broadway_df_recent['Intersecting_Street'] == intersection) & 
                             (broadway_df_recent['Complaint_Date_parsed'] == latest_date)].iloc[0]
        
        # Calculate days ago
        days_ago = (pd.Timestamp.now() - latest_date).days
        
        print(f'#{i:2d}. Broadway & {intersection}')
        print(f'     Most recent: {latest_date.strftime("%Y-%m-%d")} ({days_ago} days ago)')
        print(f'     Borough: {latest_complaint["Borough"]}')
        print(f'     Community District: {latest_complaint["Community_District"]}')
        print(f'     Complaint ID: {latest_complaint["Complaint_ID"]}')
        print(f'     Repair Status: {latest_complaint["Temp_Repair_Feasible"]}')
        
        # Check if this intersection has multiple complaints
        total_complaints = len(broadway_df_recent[broadway_df_recent['Intersecting_Street'] == intersection])
        if total_complaints > 1:
            print(f'     Total complaints: {total_complaints}')
        
        if pd.notna(latest_complaint['Temp_Repair_Date']):
            print(f'     Repair Date: {latest_complaint["Temp_Repair_Date"]}')
        else:
            print(f'     Repair Status: ❌ NOT REPAIRED')
        print()

print('\n🎯 TOP 3 MOST RECENT FOR IMMEDIATE FIELD REPORTING:')
print('='*55)

# Get top 3 most recent with full details
top_3_recent = intersection_latest.head(3)
for i, (idx, row) in enumerate(top_3_recent.iterrows(), 1):
    intersection = row['Intersecting_Street']
    latest_date = row['Complaint_Date_parsed']
    latest_complaint = broadway_df_recent[(broadway_df_recent['Intersecting_Street'] == intersection) & 
                         (broadway_df_recent['Complaint_Date_parsed'] == latest_date)].iloc[0]
    days_ago = (pd.Timestamp.now() - latest_date).days
    
    print(f'🎯 TARGET #{i}: Broadway & {intersection}')
    print(f'   Date: {latest_date.strftime("%Y-%m-%d")} ({days_ago} days ago)')
    print(f'   Location: {latest_complaint["Borough"]}, CD {latest_complaint["Community_District"]}')
    print(f'   Corner ID: {latest_complaint["CMT_Corner_ID"]}')
    print(f'   Status: {"REPAIRED" if pd.notna(latest_complaint["Temp_Repair_Date"]) else "NOT REPAIRED"}')
    print(f'   Feasible: {latest_complaint["Temp_Repair_Feasible"]}')
    print()

print('💡 FIELD REPORTING STRATEGY:')
print('='*35)
print('• Start with #1 most recent - freshest issue')
print('• Current problems more likely to find witnesses')
print('• Recent complaints = ongoing accessibility challenges') 
print('• Better chance of finding original complainants')
print('• Focus on unrepaired "feasible" complaints for policy story')


In [ ]:
# DETAILED FIELD REPORTING INFO FOR TOP 3 MOST RECENT COMPLAINTS
print('\n' + '='*80)
print('COMPLETE FIELD REPORTING DETAILS')
print('TOP 3 MOST RECENT BROADWAY COMPLAINTS')
print('='*80)

# Get top 3 most recent intersections for detailed analysis
top_3_recent_detailed = intersection_latest.head(3)

for i, (idx, row) in enumerate(top_3_recent_detailed.iterrows(), 1):
    intersection = row['Intersecting_Street']
    latest_date = row['Complaint_Date_parsed']
    
    # Get the most recent complaint details
    latest_complaint = broadway_df_recent[(broadway_df_recent['Intersecting_Street'] == intersection) & 
                         (broadway_df_recent['Complaint_Date_parsed'] == latest_date)].iloc[0]
    
    days_ago = (pd.Timestamp.now() - latest_date).days
    
    print(f'\n🎯 FIELD TARGET #{i}: Broadway & {intersection}')
    print('='*60)
    print(f'📅 COMPLAINT DATE: {latest_date.strftime("%Y-%m-%d")} ({days_ago} days ago)')
    print(f'📍 EXACT LOCATION:')
    print(f'   Address: {latest_complaint["Street_Name1"]} & {latest_complaint["Street_Name2"]}')
    print(f'   Borough: {latest_complaint["Borough"]}')
    print(f'   Community District: {latest_complaint["Community_District"]}')
    print(f'   Block: {latest_complaint["Block"]}, Lot: {latest_complaint["Lot"]}')
    print(f'   Corner ID: {latest_complaint["CMT_Corner_ID"]}')
    
    print(f'\n🆔 COMPLAINT DETAILS:')
    print(f'   Complaint ID: {latest_complaint["Complaint_ID"]}')
    print(f'   Repair Feasible: {latest_complaint["Temp_Repair_Feasible"]}')
    if pd.notna(latest_complaint['Temp_Repair_Date']):
        print(f'   Repair Date: {latest_complaint["Temp_Repair_Date"]}')
        print(f'   Repair Type: {latest_complaint["Temp_Repair_Type"]}')
        repair_days = (pd.to_datetime(latest_complaint['Temp_Repair_Date']) - latest_date).days
        print(f'   Repair Timeline: {repair_days} days from complaint to repair')
    else:
        print(f'   Repair Status: ❌ NOT REPAIRED')
        print(f'   Days without repair: {days_ago}')
    
    print(f'   Bulk Complaint: {latest_complaint["BulkComplaint"]}')
    
    # Check for other complaints at same intersection
    all_complaints_here = broadway_df_recent[broadway_df_recent['Intersecting_Street'] == intersection]
    total_complaints = len(all_complaints_here)
    
    if total_complaints > 1:
        print(f'\n📊 INTERSECTION HISTORY:')
        print(f'   Total complaints: {total_complaints}')
        date_range_start = all_complaints_here["Complaint_Date_parsed"].min()
        print(f'   Date range: {date_range_start.strftime("%Y-%m-%d")} to {latest_date.strftime("%Y-%m-%d")}')
        
        # Show repair success rate
        repaired_count = len(all_complaints_here[all_complaints_here['Temp_Repair_Date'].notna()])
        repair_rate = (repaired_count / total_complaints) * 100
        print(f'   Repair rate: {repaired_count}/{total_complaints} ({repair_rate:.0f}%)')
    
    # Subway/transportation info based on location
    if latest_complaint['Community_District'] == 101:
        print(f'\n🚇 FIELD ACCESS INFO:')
        print(f'   Subway: 4/5/6 at Fulton St or R/W at City Hall')
        print(f'   Neighborhood: Lower Manhattan/Financial District')
    elif latest_complaint['Community_District'] == 107:
        print(f'\n🚇 FIELD ACCESS INFO:') 
        print(f'   Subway: 1/2/3 at 72nd, 79th, 86th, or 96th St')
        print(f'   Neighborhood: Upper West Side')
    elif latest_complaint['Community_District'] == 109:
        print(f'\n🚇 FIELD ACCESS INFO:') 
        print(f'   Subway: 1 at 103rd, 110th, or 116th St')
        print(f'   Neighborhood: Manhattan Valley/Morningside Heights')
    
    print(f'\n💡 WHY THIS IS GOOD FOR REPORTING:')
    print(f'   • Recent complaint = current accessibility issue')
    print(f'   • {days_ago} days ago = likely still problematic')
    print(f'   • Fresh issue = better chance finding witnesses')
    if latest_complaint['Temp_Repair_Feasible'] == 'No':
        print(f'   • "Not feasible" status = ongoing city policy story')
    if pd.isna(latest_complaint['Temp_Repair_Date']):
        print(f'   • Unrepaired = active accessibility barrier')
    if latest_complaint['Temp_Repair_Feasible'] == 'Yes' and pd.isna(latest_complaint['Temp_Repair_Date']):
        print(f'   • Feasible but unrepaired = city responsiveness story')

# Overall recommendation
print(f'\n🏆 RECOMMENDATION FOR FIELD REPORTING:')
print('='*50)
top_intersection_detailed = top_3_recent_detailed.iloc[0]['Intersecting_Street']
top_date_detailed = top_3_recent_detailed.iloc[0]['Complaint_Date_parsed']
top_days_detailed = (pd.Timestamp.now() - top_date_detailed).days

print(f'START WITH: Broadway & {top_intersection_detailed}')
print(f'REASON: Most recent complaint ({top_days_detailed} days ago)')
print(f'STRATEGY: Current issue = best story potential')
print(f'TIMING: Visit during business hours for witness interviews')

print(f'\n📱 FIELD REPORTING CHECKLIST:')
print('='*35)
print('□ Check current ramp condition at corner')
print('□ Photo document accessibility barriers')
print('□ Interview pedestrians using intersection')
print('□ Contact local businesses for comment')
print('□ Verify complaint status with NYC 311')
print('□ Ask Community Board about ongoing issues')
print('□ Get NYC DOT response about repair determination')

print(f'\n📰 STORY ANGLES:')
print('='*20)
print('• Why recent complaints remain unrepaired')
print('• "Not feasible" determinations - what makes them unfixable?')  
print('• Current accessibility barriers in high-traffic areas')
print('• City responsiveness to accessibility complaints')
print('• Real-time impact on disabled pedestrians')
